## Getting started with LIME: A explainer for a black box model

This notebook shows example of how LIME can be applied to different models with different kinds of data.

LIME makes models more **trustfull** and **interpretabel**. 

Before running this notebook make sure you have all dependencies installed:

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections

import numpy as np
import pandas as pd

from IPython.display import Image

import tensorflow as tf
print('This code was created using TensorFlow version 1.4.1, probably TensorFlow >= v1.0 is fine.')
print('Your TensorFlow version:', tf.__version__)

This code was created using TensorFlow version 1.4.1, probably TensorFlow >= v1.0 is fine.
Your TensorFlow version: 1.5.0


## How does it works?

Intuitively, an explanation is a local linear approximation of the model's behaviour. While the model may be very complex globally, it is easier to approximate it around the vicinity of a particular instance. While treating the model as a black box, we perturb the instance we want to explain and learn a sparse linear model around it, as an explanation.

The figure below illustrates the intuition for this procedure. The model's decision function is represented by the blue/pink background, and is clearly nonlinear. The bright red cross is the instance being explained (let's call it X). We sample instances around X, and weight them according to their proximity to X (weight here is indicated by size). We then learn a linear model (dashed line) that approximates the model well in the vicinity of X, but not necessarily globally. For more information, check [the paper](https://arxiv.org/abs/1602.04938).

![](https://raw.githubusercontent.com/marcotcr/lime/master/doc/images/lime.png)

## Example: Census Dataset (Tabular data)

We'll use the [Adult dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/) from the 1990 US Census. Our task is to predict whether an individual has an income over $50,000 / year (a classification problem), based attributes such as their age and occupation. This is a generic problem with a variety of numeric and categorical attributes - which makes it useful for demonstration purposes.

This code is based on the code present [here](https://github.com/random-forests/tensorflow-workshop/blob/master/examples/07_structured_data.ipynb).

In [2]:
census_train_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
census_test_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'
census_train_path = tf.contrib.keras.utils.get_file('census.train', census_train_url)
census_test_path = tf.contrib.keras.utils.get_file('census.test', census_test_url)

There are a lot of features in this dataset, **this makes hard to understand and explain the model output**

In [3]:
column_names = [
  'age', 'workclass', 'fnlwgt', 'education', 'education-num',
  'marital-status', 'occupation', 'relationship', 'race', 'gender',
  'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
  'income'
]

### Load the dataset using Pandas


In [9]:
# Notes
# 1) We provide the header from above.
# 2) The test file has a line we want to disgard at the top, so we include the parameter 'skiprows=1'
census_train = pd.read_csv(census_train_path, index_col=False, names=column_names) 
census_test = pd.read_csv(census_test_path, skiprows=1, index_col=False, names=column_names) 

# Drop any rows that have missing elements
# There are other ways to handle missing data, but we'll
# take the simplest approach here.
census_train = census_train.dropna(how="any", axis=0)
census_test = census_test.dropna(how="any", axis=0)

# Separate the label we want to predict into its own object 
# At the same time, we'll convert it into true/false to fix the formatting error
census_train_label = census_train.pop('income').apply(lambda x: ">50K" in x)
census_test_label = census_test.pop('income').apply(lambda x: ">50K" in x)

### Treat categorical features

In [10]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
census_train['workclass'] = lb_make.fit_transform(census_train['workclass'])
census_test['workclass'] = lb_make.fit_transform(census_test['workclass'])

categorical_names = {
    'workclass': lb_make.classes_
}

print(categorical_names)

{'workclass': array([' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private',
       ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
      dtype=object)}


## Training a simple classifier

In [6]:
def create_train_input_fn(): 
    return tf.estimator.inputs.pandas_input_fn(
        x=census_train,
        y=census_train_label, 
        batch_size=32,
        num_epochs=None, # Repeat forever
        shuffle=True)

def create_test_input_fn():
    return tf.estimator.inputs.pandas_input_fn(
        x=census_test,
        y=census_test_label, 
        num_epochs=1, # Just one epoch
        shuffle=False) # Don't shuffle so we can compare to census_test_labels later

feature_columns = []

age = tf.feature_column.numeric_column('age')
feature_columns.append(age)

workclass = tf.feature_column.categorical_column_with_vocabulary_list('workclass',
                                                                      vocabulary_list=[x for x in list(set(census_train['workclass']))])
feature_columns.append(workclass)

'''
age_buckets = tf.feature_column.bucketized_column(
    tf.feature_column.numeric_column('age'), 
    boundaries=[31, 46, 60, 75, 90] # specify the ranges
)

feature_columns.append(age_buckets)

education = tf.feature_column.categorical_column_with_vocabulary_list(
    "education", [
        "Bachelors", "HS-grad", "11th", "Masters", "9th",
        "Some-college", "Assoc-acdm", "Assoc-voc", "7th-8th",
        "Doctorate", "Prof-school", "5th-6th", "10th", "1st-4th",
        "Preschool", "12th"
    ])

feature_columns.append(education)

# A categorical feature with a possibly large number of values
# and the vocabulary not specified in advance.
native_country = tf.feature_column.categorical_column_with_hash_bucket('native-country', 1000)
feature_columns.append(native_country)

age_cross_education = tf.feature_column.crossed_column(
    [age_buckets, education],
    hash_bucket_size=int(1e4) # Using a hash is handy here
)
feature_columns.append(age_cross_education)
'''

train_input_fn = create_train_input_fn()
estimator = tf.estimator.LinearClassifier(feature_columns, model_dir='graphs/linear3', n_classes=2)
estimator.train(train_input_fn, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_save_summary_steps': 100, '_num_worker_replicas': 1, '_model_dir': 'graphs/linear3', '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcfa11a1978>, '_is_chief': True, '_task_id': 0, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_tf_random_seed': None, '_service': None, '_keep_checkpoint_max': 5}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into graphs/linear3/model.ckpt.
INFO:tensorflow:step = 1, loss = 22.18071
INFO:tensorflow:global_step/sec: 293.109
INFO:tensorflow:step = 101, loss = 14.63991 (0.346 sec)
INFO:tensorflow:global_step/sec: 317.276
INFO:tensorflow:step = 201, loss = 16.94721 (0.317 sec)
INFO:tensorflow:global_step/sec: 331.611
INFO:tensorflow:step = 301, l

In [11]:
test_input_fn = create_test_input_fn()
estimator.evaluate(test_input_fn)

INFO:tensorflow:Starting evaluation at 2018-02-08-03:31:32
INFO:tensorflow:Restoring parameters from graphs/linear3/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-02-08-03:31:33
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.7673976, accuracy_baseline = 0.76377374, auc = 0.69676805, auc_precision_recall = 0.3723111, average_loss = 0.5125317, global_step = 1000, label/mean = 0.23622628, loss = 65.19163, prediction/mean = 0.20704338


{'accuracy': 0.7673976,
 'accuracy_baseline': 0.76377374,
 'auc': 0.69676805,
 'auc_precision_recall': 0.3723111,
 'average_loss': 0.5125317,
 'global_step': 1000,
 'label/mean': 0.23622628,
 'loss': 65.19163,
 'prediction/mean': 0.20704338}

In [21]:
def predict_fn(x):
    def create_test_input_fn():
        return tf.estimator.inputs.numpy_input_fn(
            x={'age': np.array(x[:, 0]), 'workclass': np.array(map(int, x[:, 1]))},
            y=None, 
            num_epochs=1, # Just one epoch
            shuffle=False) # Don't shuffle so we can compare to census_test_labels later

    preds = []
    tf_prediction = estimator.predict(create_test_input_fn())
    for p in tf_prediction:
        preds.append(p['probabilities'])
    return np.array(preds)

In [22]:
# reinitialize the input function
test_input_fn = create_test_input_fn()

predictions = estimator.predict(test_input_fn)
i = 0
for prediction in predictions:
    true_label = census_test_label[i]
    predicted_label = prediction['class_ids'][0]
    # Uncomment the following line to see probabilities for individual classes
    # print(prediction) 
    print("Example %d. Actual: %d, Predicted: %d" % (i, true_label, predicted_label))
    i += 1
    if i == 10: break

INFO:tensorflow:Restoring parameters from graphs/linear3/model.ckpt-1000
Example 0. Actual: 0, Predicted: 0
Example 1. Actual: 0, Predicted: 0
Example 2. Actual: 1, Predicted: 0
Example 3. Actual: 1, Predicted: 0
Example 4. Actual: 0, Predicted: 0
Example 5. Actual: 0, Predicted: 0
Example 6. Actual: 0, Predicted: 0
Example 7. Actual: 1, Predicted: 0
Example 8. Actual: 0, Predicted: 0
Example 9. Actual: 0, Predicted: 0


In [23]:
import lime
import lime.lime_tabular

feature_names = ['age', 'workclass']

train = census_train.filter(['age','workclass'], axis=1)
test = census_test.filter(['age','workclass'], axis=1)

categorical_features = [1]
explainer = lime.lime_tabular.LimeTabularExplainer(train.values, 
                                                   feature_names=feature_names,
                                                   class_names=['income < 50', 'income >= 50'],
                                                   categorical_features=categorical_features, verbose=True, mode='classification',
                                                   discretize_continuous=True)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [25]:
i = 10
exp = explainer.explain_instance(test.values[i], predict_fn)

IndexError: tuple index out of range

## Example: Text

## Example: Images

## Building a interpretable model to classify Fake News

## 